<a href="https://colab.research.google.com/github/iocariz/stock_market_analytics/blob/main/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# install main library YFinance
!pip install --upgrade yfinance

In [7]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import time
from datetime import date

In [8]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=24
Period for indexes: 1954-04-24 to 2024-04-24 


# Question 1: [Macro] Average growth of GDP in 2023
## What is the average growth (in %) of GDP in 2023?

In [9]:
# Real Potential Gross Domestic Product (GDPC1), QUARTERLY
# https://fred.stlouisfed.org/series/GDPC1
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)
gdpc1.tail()

,GDPC1
DATE,
2022-10-01,21989.981
2023-01-01,22112.329
2023-04-01,22225.350
2023-07-01,22490.692
2023-10-01,22679.255


In [10]:
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1 / gdpc1.GDPC1.shift(4) - 1
gdpc1['gdpc1_us_yoy'] = round(gdpc1['gdpc1_us_yoy'] * 100, 1)

#Calculating Average Growth 2023
gdpc1_2023 = gdpc1.loc['2023']
average_growth_2023 = gdpc1_2023['gdpc1_us_yoy'].mean()

print(f"The Average Yearly Growth for 2023 is {average_growth_2023}% ")


The Average Yearly Growth for 2023 is 2.525% 


# Question 2. [Macro] Inverse "Treasury Yield"

## Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

In [11]:
start = '2000-01-01'

dgs2 = pdr.DataReader("DGS2", "fred", start=start)
dgs10 = pdr.DataReader("DGS10", "fred", start=start)

In [12]:
# Joining Data
interest_rates = dgs10.join(dgs2, how='inner', lsuffix='_dgs10', rsuffix='_dgs2')

In [13]:
interest_rates

,DGS10,DGS2
DATE,,
2000-01-03,6.58,6.38
2000-01-04,6.49,6.30
2000-01-05,6.62,6.38
2000-01-06,6.57,6.35
2000-01-07,6.52,6.31
...,...,...
2024-04-16,4.67,4.97
2024-04-17,4.59,4.93
2024-04-18,4.64,4.98


In [14]:
# Calculating Difference
interest_rates['diff'] = interest_rates['DGS10'] - interest_rates['DGS2']

In [15]:
# Finding Minimum
min_diff_since_2000 = round(interest_rates['diff'].min(),1)

In [16]:
print("Minimum value of (dgs10 - dgs2) since 2000:", min_diff_since_2000)

Minimum value of (dgs10 - dgs2) since 2000: -1.1


# Question 3. [Index] Which Index is better recently?

## Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

In [18]:
# Start- and Enddate
start_date = '2019-04-09'
end_date = '2024-04-09'

# Loading Data S&P 500
spx_index = yf.download(tickers="^GSPC", start=start_date, end=end_date, progress=False)['Adj Close']

# Loading Data IPC Mexico
ipc_index = yf.download(tickers="^MXX", start=start_date, end=end_date, progress=False)['Adj Close']

# Calculation Relation between Endprice and Startprice
spx_growth_ratio = spx_index[-1] / spx_index[0]
ipc_growth_ratio = ipc_index[-1] / ipc_index[0]

# Converting into Percentage
spx_growth_percentage = (spx_growth_ratio - 1) * 100
ipc_growth_percentage = (ipc_growth_ratio - 1) * 100

# Comparing
if spx_growth_percentage > ipc_growth_percentage:
    better_index = 'S&P 500'
    growth_percentage = round(spx_growth_percentage)
else:
    better_index = 'IPC Mexico'
    growth_percentage = round(ipc_growth_percentage)

print(f"The better performing index recently is {better_index} with a growth of {growth_percentage}% over the last five years.")



The better performing index recently is S&P 500 with a growth of 81% over the last five years.


In [19]:
spx_index

Date
2019-04-09    2878.199951
2019-04-10    2888.209961
2019-04-11    2888.320068
2019-04-12    2907.409912
2019-04-15    2905.580078
                 ...     
2024-04-02    5205.810059
2024-04-03    5211.490234
2024-04-04    5147.209961
2024-04-05    5204.339844
2024-04-08    5202.390137
Name: Adj Close, Length: 1258, dtype: float64

# Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks

## Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

In [22]:
tickers = ['2222.SR', 'BRK-B', 'AAPL', 'MSFT', 'GOOG', 'JPM']

# Loading Data
data = yf.download(tickers, start="2023-01-01", end="2023-12-31", progress=False)

# Calculating max und min Adj. Close Price for 2023
max_prices = data['Adj Close'].max()
min_prices = data['Adj Close'].min()

# Calculation Relation from (max - min) / max for every Stock
range_ratios = ((max_prices - min_prices) / max_prices).round(2)

# Finding Stock with biggest Ratio
max_range_ratio_stock = range_ratios.idxmax()
max_range_ratio_value = range_ratios.max()

print(f"The stock with the largest 52-weeks range ratio in 2023 is {max_range_ratio_stock} with a ratio of {max_range_ratio_value}")

The stock with the largest 52-weeks range ratio in 2023 is MSFT with a ratio of 0.42


# Question 5. [Stocks] Dividend Yield
## Find the largest dividend yield for the same set of stocks

In [23]:
# Load dividend data for 2023
dividends_data = yf.download(tickers, start="2023-01-01", end="2023-12-31", progress=False, actions=True)

# Sum up dividends paid in 2023 per company
total_dividends = dividends_data['Dividends'].sum()

# Get the closing prices (Adj.Close) at the last trading day of the year
last_trading_day_prices = dividends_data.iloc[-1]['Adj Close']

# Calculate dividend yield for each company
dividend_yields = (total_dividends / last_trading_day_prices) * 100

# Find the maximum dividend yield
max_dividend_yield = dividend_yields.max()

# Round to 1 digit after the decimal point
max_dividend_yield_rounded = round(max_dividend_yield, 1)

print(f"The largest dividend yield for the same set of stocks in 2023 is {max_dividend_yield_rounded}%")

The largest dividend yield for the same set of stocks in 2023 is 2.4%
